In [1]:
!git clone https://github.com/sahal-mulki/geo-llm/

Cloning into 'geo-llm'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 40 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (40/40), 1.57 MiB | 3.50 MiB/s, done.


In [2]:
import pandas as pd

In [3]:
def prompt_generator(addresses):
    prompt = (
        "Guess where this image was taken, get creative."
        "Here's some AI-generated addresses that could be wrong to get you started. Choose the best out of these if you're uncertain about the image:\n"
        f"1. {addresses[0]}\n"
        f"2. {addresses[1]}\n"
        f"3. {addresses[2]}\n"
        f"4. {addresses[3]}\n"
        f"5. {addresses[4]}"
    )
    
    return prompt

def image_info_finder(image_name, dataset):
    
    if "yfcc" in dataset:
        df = pd.read_csv("/kaggle/working/geo-llm/geoclip_predictions_yfcc4k.csv")
        addresses = [df[df["Image"] == image_name]["Prediction 1 Address"].values[0], df[df["Image"] == image_name]["Prediction 2 Address"].values[0], df[df["Image"] == image_name]["Prediction 1 Address"].values[0], df[df["Image"] == image_name]["Prediction 3 Address"].values[0], df[df["Image"] == image_name]["Prediction 4 Address"].values[0], df[df["Image"] == image_name]["Prediction 5 Address"].values[0]]
    elif "im2gps" in dataset:
        df = pd.read_csv("/kaggle/working/geo-llm/geoclip_predictions_im2gps3k.csv")
        addresses = [df[df["Image"] == image_name]["Prediction 1 Address"].values[0], df[df["Image"] == image_name]["Prediction 2 Address"].values[0], df[df["Image"] == image_name]["Prediction 1 Address"].values[0], df[df["Image"] == image_name]["Prediction 3 Address"].values[0], df[df["Image"] == image_name]["Prediction 4 Address"].values[0], df[df["Image"] == image_name]["Prediction 5 Address"].values[0]]
        
    return addresses



In [14]:
print(prompt_generator(image_info_finder("10003206806.jpg", "yfcc")))

Guess where this image was taken, get creative.Here's some AI-generated addresses that could be wrong to get you started. Choose the best out of these if you're uncertain about the image:
1. Charente-Maritime, Nouvelle-Aquitaine, Metropolitan France, France
2. Dordogne, Nouvelle-Aquitaine, Metropolitan France, France
3. Charente-Maritime, Nouvelle-Aquitaine, Metropolitan France, France
4. Vaucluse, Provence-Alpes-Côte d'Azur, Metropolitan France, France
5. Vosges, Grand Est, Metropolitan France, France


In [15]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [16]:
import base64

In [20]:
df = pd.read_csv('/kaggle/input/llms-for-image-geolocation-benchmarks/archive(2)/yfcc4k/yfcc4k/yfcc4k.csv')

from tqdm import tqdm

jsonl_reqs_for_im2gps = []
# Iterate over the rows of the DataFrame
for index, row in tqdm(df.iterrows(), total=1000):
    
    prompt = (
        "You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. "
        "By the end of your investigation you have to have at least a rough guess of the image's location. "
        "DECIDE ON ONE FINAL LOCATION, EVEN IF IT'S WRONG, JUST GUESS. "
             )
    
    try:
        encoded = encode_image("/kaggle/input/llms-for-image-geolocation-benchmarks/archive(2)/yfcc4k/yfcc4k/yfcc4k/" + row["Image"])

        boilerplate = {
          "custom_id": "yfcc" + row["Image"],
          "method": "POST",
          "url": "/v1/chat/completions",
          "body": {
            "model": "gpt-4o",
            "messages": [
              {
                "role": "system",
                "content": prompt
              },
              {
                "role": "user",
                "content": [
                {
                  "type": "text",
                  "text": prompt_generator(image_info_finder(row["Image"], "yfcc"))
                },
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded}"
                  }
                }
              ]
              }
            ],
            "max_tokens": 1000
          }
        }

        jsonl_reqs_for_im2gps.append(boilerplate)
    except TypeError:
        pass

100%|██████████| 1000/1000 [01:02<00:00, 16.00it/s]


In [21]:
len(jsonl_reqs_for_im2gps)

1000

In [22]:
json_strings = [json.dumps(req) for req in tqdm(jsonl_reqs_for_im2gps)]

100%|██████████| 1000/1000 [00:00<00:00, 1081.53it/s]


In [23]:
with open('yfcc_4k_batch_api_requests.jsonl', 'w') as f:
    for json_str in tqdm(json_strings):
        f.write(json_str + '\n')

100%|██████████| 1000/1000 [00:00<00:00, 4366.41it/s]
